In [91]:
import torch
import torch.nn as nn
import torch.optim as optim

In [92]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [93]:
class PreContextRewardModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers=1):
    super(PreContextRewardModel, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    # Embedding layer
    self.embedding = nn.Embedding(input_size, hidden_size)

    # GRU layer
    self.gru = nn.GRU(hidden_size, hidden_size, num_layers, batch_first=True)

    # Output layer
    self.fc = nn.Linear(hidden_size, output_size)
    # self.softmax = nn.Softmax(dim=1)

  def forward(self, x, hidden):
    # Embed the input tokens
    x = self.embedding(x)

    # Pass through GRU
    out, hidden = self.gru(x, hidden)

    # Take the output of the last time step
    out = out[:, -1, :]

    # Pass through fully connected layer
    out = self.fc(out)

    # Apply softmax to get probabilities
    # out = self.softmax(out)

    return out, hidden

  def init_hidden(self, batch_size):
    return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
import json, os

In [96]:
dataset_path = "/content/drive/MyDrive/ECE570/data/"

In [97]:
def create_duet_pairs(parts):
  """Create duet pairs from the four parts."""
  duet_pairs = [
    (parts["soprano"], parts["alto"]),
    (parts["soprano"], parts["tenor"]),
    (parts["soprano"], parts["bass"]),
    (parts["alto"], parts["tenor"]),
    (parts["alto"], parts["bass"]),
    (parts["tenor"], parts["bass"]),
  ]
  return duet_pairs

In [98]:
def tokenize_part(part):
  """Convert a part into a sequence of tokenized notes."""
  tokens = []
  for n in part:
    if n == "hold":
      tokens.append(128)
    else:
      tokens.append(n)
  return tokens

In [99]:
duet_data = []
for filename in os.listdir(dataset_path):
  if filename.endswith(".json"):  # Check if the file is a JSON file
    filepath = os.path.join(dataset_path, filename)
    with open(filepath, "r", encoding="utf-8") as file:
      chorale_data = json.load(file)
      duet_pairs = create_duet_pairs(chorale_data)
      for human_part, machine_part in duet_pairs:
        human_tokens = tokenize_part(human_part)
        machine_tokens = tokenize_part(machine_part)
        duet_data.append((human_tokens, machine_tokens))

In [100]:
def prepare_training_data(duet_data, window_size=16):
  """Prepare training data for the reward model."""
  inputs = []
  targets = []
  for human_tokens, machine_tokens in duet_data:
    # Ensure both parts have the same length
    min_length = min(len(human_tokens), len(machine_tokens))
    human_tokens = human_tokens[:min_length]
    machine_tokens = machine_tokens[:min_length]
    # Create input-target pairs
    for i in range(window_size, min_length):
      input_seq = human_tokens[i-window_size:i]
      target = machine_tokens[i]
      inputs.append(input_seq)
      targets.append(target)

  return inputs, targets

In [106]:
from sklearn.model_selection import train_test_split

# Prepare dataset
inputs, targets = prepare_training_data(duet_data)

# Split into train and test sets (e.g., 80% train, 20% test)
train_inputs, test_inputs, train_targets, test_targets = train_test_split(
    inputs, targets, test_size=0.2, random_state=42
)


In [107]:
# Hyperparameters
input_size = 129  # Number of unique tokens (MIDI pitches + rest)
hidden_size = 256
output_size = 129  # Same as input_size
num_layers = 2
learning_rate = 0.001
num_epochs = 10
batch_size = 32

In [108]:
# Initialize model, loss, and optimizer
model = PreContextRewardModel(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [109]:
train_inputs = torch.tensor(train_inputs, dtype=torch.long).to(device)
train_targets = torch.tensor(train_targets, dtype=torch.long).to(device)

In [110]:
# Training loop
for epoch in range(num_epochs):
  model.train()

  for i in range(0, len(train_inputs), batch_size):
    # Get batch
    batch_inputs = train_inputs[i:i+batch_size]
    batch_targets = train_targets[i:i+batch_size]
    # Convert to tensors
    # batch_inputs = torch.tensor(batch_inputs, dtype=torch.long).to(device)
    # batch_targets = torch.tensor(batch_targets, dtype=torch.long).to(device)

    # Dynamically initialize hidden state
    hidden = model.init_hidden(len(batch_inputs))
    # Forward pass
    hidden = hidden.detach()  # Detach hidden state to avoid backprop through time
    train_outputs, hidden = model(batch_inputs, hidden)
    # print(f"Outputs shape: {train_outputs.shape}")
    # print(f"Batch targets shape: {batch_targets.shape}")
    # print(train_outputs)
    # print(batch_targets)

    # Compute loss
    loss = criterion(train_outputs, batch_targets)
    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()

    # Apply gradient clipping
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    optimizer.step()

  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.7925
Epoch [2/10], Loss: 0.7247
Epoch [3/10], Loss: 0.6686
Epoch [4/10], Loss: 0.6975
Epoch [5/10], Loss: 0.6685
Epoch [6/10], Loss: 0.7708
Epoch [7/10], Loss: 0.6706
Epoch [8/10], Loss: 0.7857
Epoch [9/10], Loss: 0.5735
Epoch [10/10], Loss: 0.5746


In [111]:
model_path = "/content/drive/MyDrive/ECE570/models/pre_context_reward_model.pth"
torch.save(model.state_dict(), model_path)

In [113]:
test_inputs = torch.tensor(test_inputs, dtype=torch.long).to(device)
test_targets = torch.tensor(test_targets, dtype=torch.long).to(device)

In [114]:
def evaluate_model(model, test_inputs, test_targets):
    model.eval()  # Set to evaluation mode (disables dropout, etc.)
    with torch.no_grad():  # Disable gradient computation
        hidden = model.init_hidden(len(test_inputs))  # Adjust if needed
        test_outputs, _ = model(test_inputs, hidden)

    # Compute loss
    loss = criterion(test_outputs, test_targets)
    print(f"Test Loss: {loss.item()}")

    # Convert outputs to probabilities
    probs = torch.softmax(test_outputs, dim=1)
    predicted = torch.argmax(probs, dim=1)

    # Compute accuracy
    accuracy = (predicted == test_targets).float().mean().item()
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Run evaluation
evaluate_model(model, test_inputs, test_targets)


Test Loss: 1.069266438484192
Test Accuracy: 72.42%
